# Import dataset

In [89]:
from tensorflow import keras
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

train_set = tfds.load('imdb_reviews', split='train', as_supervised=True)
test_set = tfds.load('imdb_reviews', split='test', as_supervised=True)

# Prepare tokens

In [90]:
max_text_length = 0

X_train = []
y_train = []
for i, j in train_set:
    i = str(i.numpy())
    max_text_length = max(max_text_length, len(i))
    X_train.append(i)
    y_train.append(int(j))

tokenizer = keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

# Build the RNN
We need to use an RNN to help predict positive/negative reviews because the order of words obviously matters in sentences. An embedding layer is needed to convert sequences of tokens into sequences of vectors that can be easily understood by the RNN layer.

In [91]:
model = keras.Sequential()
model.add(keras.layers.Embedding(10001, 128, mask_zero=True))
model.add(keras.layers.SimpleRNN(100, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[keras.metrics.BinaryAccuracy()])

# Train the model

In [92]:
epochs = 10

X_train = tokenizer.texts_to_sequences(X_train)
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_text_length, padding='post')

X_train = np.array(X_train)
y_train = np.array(y_train)

model.fit(X_train, y_train, epochs=epochs, validation_split=0.1)

Epoch 1/10
  4/704 [..............................] - ETA: 6:03:17 - loss: 0.6867 - binary_accuracy: 0.6452